In [1]:
HF_Token = 'YOUR HUGGINGFACE TOKEN'
from huggingface_hub import login
login(token=HF_Token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir content/drive/MyDrive/StanceEval

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘content/drive/MyDrive/StanceEval’: No such file or directory


In [3]:
!git clone https://github.com/NoraAlt/Mawqif-Arabic-Stance.git

fatal: destination path 'Mawqif-Arabic-Stance' already exists and is not an empty directory.


In [4]:
!pip install peft
!pip install trl
!pip install bitsandbytes

In [5]:
import pandas as pd
import numpy as np

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset, Dataset
from trl import SFTTrainer
import torch

In [7]:
train = pd.read_csv('Mawqif-Arabic-Stance/Data/Mawqif_AllTargets_Train.csv')
test = pd.read_csv('Mawqif-Arabic-Stance/Data/Mawqif_AllTargets_Blind Test.csv')

In [8]:
train = train.sort_values(by=['target'])
train = train.reset_index()

In [9]:
mylist = []
inst = '<s> [INST] <<SYS>> You are an expert Arabic stance analyzer! <</SYS>>  \
Consider the subsequent Arabic passage which discusses  '
inst2 =  ' and determine if it is positive, neutral, or negative, and return the answer as \
            the corresponding stance label "positive" or "neutral" or "negative".\n'
for index, row in train.iterrows():
  string = inst + row['target'] + inst2  + row['text'] + ' [/INST]\n' + row['sentiment'] + '</s>'
  mylist.append(string)

In [10]:
import random
random.shuffle
random.shuffle(mylist)

In [11]:
eval_list = [mylist[i] for i in range(len(mylist)) if i % 5 == 0]
train_list = [mylist[i] for i in range(len(mylist)) if i % 5 > 0]

In [12]:
train_dataset = Dataset.from_dict({"text": train_list})
eval_dataset = Dataset.from_dict({"text": eval_list})
full_dataset = Dataset.from_dict({"text": mylist})

In [13]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=compute_dtype,
            bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map={"": 0}, quantization_config=bnb_config)
model = prepare_model_for_kbit_training(model)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, add_eos_token=True,device='cuda')
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "left"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
peft_config = LoraConfig(
            lora_alpha=16,
            lora_dropout=0.05,
            r=16,
            bias="none",
            task_type="SEQ_CLS",
            target_modules= ["down_proj","up_proj","gate_proj"]
)

In [16]:
peft_config = LoraConfig(
            lora_alpha=16,
            lora_dropout=0.05,
            r=64,
            bias="none",
            task_type="SEQ_CLS",
            target_modules= ["q_proj","up_proj","o_proj","k_proj","down_proj","gate_proj","v_proj"]
)

In [17]:
training_arguments = TrainingArguments(
        output_dir="./results/",
        evaluation_strategy="steps",
        optim="paged_adamw_8bit",
        save_steps=50,
        log_level="debug",
        logging_steps=50,
        learning_rate=1e-4,
        eval_steps=50,
        fp16=True,
        do_eval=True,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=100,
        #gradient_accumulation_steps=1,
        warmup_steps=10,
        max_steps=5600,
        lr_scheduler_type="linear"
)

In [18]:
tokenizer.pad_token = tokenizer.eos_token
trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=120,
        tokenizer=tokenizer,
        args=training_arguments
)

trainer.train()

Map:   0%|          | 0/2801 [00:00<?, ? examples/s]

Map:   0%|          | 0/701 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend
Currently training with a batch size of: 1
***** Running training *****
  Num examples = 2,801
  Num Epochs = 2
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 5,600
  Number of trainable parameters = 167,772,160
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.util

Step,Training Loss,Validation Loss
50,2.329200,1.515919
100,1.294900,1.426506
150,1.361800,1.416085
200,1.345300,1.399704
250,1.358000,1.378971
300,1.294800,1.386693
350,1.319500,1.370785
400,1.315300,1.352193
450,1.294700,1.349688
500,1.299800,1.343619


***** Running Evaluation *****
  Num examples = 701
  Batch size = 100
Saving model checkpoint to ./results/checkpoint-50
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e5e23bbe8e749ef0efcf16cad411a7d23bd23298/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,

TrainOutput(global_step=5600, training_loss=1.1316608599254063, metrics={'train_runtime': 5407.6137, 'train_samples_per_second': 1.036, 'train_steps_per_second': 1.036, 'total_flos': 2.956811832754176e+16, 'train_loss': 1.1316608599254063, 'epoch': 1.9992859692966798})

In [19]:
test['sentiment'] = ''

In [ ]:
#test['target_msa'] = ''
for index, row in test.iterrows():
  #if len(row['target_msa']) > 1:
  #  continue
  print(index)
  prompt = inst + row['target'] + inst2  + row['text'] + ' [/INST]\n'
  tokenized_input = tokenizer(prompt, return_tensors="pt")
  input_ids = tokenized_input["input_ids"].cuda()

  generation_output = model.generate(
        input_ids=input_ids,
        num_beams=6,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=30
  )
  ans = tokenizer.decode(generation_output.sequences[0], skip_special_tokens=True).split('[/INST]')[1]
  ans = ans.split('</s>')[0].strip().replace(' و ', ' و')
  test.at[index, 'sentiment'] = ans
  #print(row['source_da'], ans)
  if index % 10 == 0:
    test.to_csv('drive/MyDrive/StanceEval/test.csv',index=False)
    test['sentiment'].to_csv('drive/MyDrive/StanceEval/predictions.csv',index=False)
    #break
test.to_csv('drive/MyDrive/StanceEval/test.csv',index=False)
test['sentiment'].to_csv('drive/MyDrive/StanceEval/predictions.csv',index=False)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

174


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


175


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


176


In [ ]:
from google.colab import runtime
runtime.unassign()